# Imports

In [4]:
#data stuff
import pandas as pd

#Modelling
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction import text

#NLP
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import gensim

#other
from tqdm import tqdm_notebook

C:\Users\steph\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


# Formatting X

In [41]:
files = ['@TotalTrafficMIA',
         '@TrafficAlertSFL', '@nbc6', '@CBSMiami', '@wsvn', '@MiamiHerald', '@SunSentinel', '@miaminewsnow',
         '@BeachNewsAlerts', '@WPLGLocal10', 
       '@MiamiBeachPD', '@miamibeachnews', '@FLTurnpikeSFL']

In [65]:
datum = []

for filename in tqdm_notebook(files):
    data = pd.read_csv(f'Datasets/siteloop_{filename}.csv')
    data.drop(columns=['User','User_ID','Geo'], inplace = True)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data.assign(Date=data['Date'].dt.round('H'))['Date']
    times = []
    times.append(data['Date'][0])

    for i in data.index:
        time = data['Date'][i]
        if time != times[len(times)-1]:
            times.append(time)

    dic = {'time': [], f'{filename}': []}

    for hour in times:
        total = ''
        tweets = list(data[data['Date'] == hour]['Tweet'])
        for twit in tweets:
            total += str(twit)
        dic['time'].append(hour)
        dic[f'{filename}'].append(total)
    exec(f'data_{filename[1:]} = pd.DataFrame.from_dict(dic)')
    
    exec(f'data_{filename[1:]}.set_index("time", inplace=True)')
    exec(f'data_{filename[1:]}.sort_index(inplace=True)')
    
    exec(f'datum.append(data_{filename[1:]})')
df = pd.concat(datum, axis=1)
df.fillna('', inplace=True)

In [77]:
df['ALL']=  df['@TotalTrafficMIA'] + df['@TrafficAlertSFL'] + df['@nbc6'] + df['@CBSMiami'] + df['@wsvn'] +\
            df['@MiamiHerald'] +  df['@SunSentinel'] + df['@miaminewsnow'] + df['@BeachNewsAlerts'] + df['@WPLGLocal10'] +\
            df['@MiamiBeachPD'] + df['@miamibeachnews'] + df['@FLTurnpikeSFL']

# Format Y

In [80]:
targets = ['@fl511_i95', '@fl511_southeast', '@fl511_95EXPRESS']

In [82]:
datum = []

for filename in tqdm_notebook(targets):
    data = pd.read_csv(f'Datasets/siteloop_{filename}.csv')
    data.drop(columns=['User','User_ID','Geo'], inplace = True)
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date'] = data.assign(Date=data['Date'].dt.round('H'))['Date']
    times = []
    times.append(data['Date'][0])

    for i in data.index:
        time = data['Date'][i]
        if time != times[len(times)-1]:
            times.append(time)

    dic = {'time': [], f'{filename}': []}

    for hour in times:
        total = ''
        tweets = list(data[data['Date'] == hour]['Tweet'])
        for twit in tweets:
            total += str(twit)
        dic['time'].append(hour)
        dic[f'{filename}'].append(total)
    exec(f'data_{filename[1:]} = pd.DataFrame.from_dict(dic)')
    
    exec(f'data_{filename[1:]}.set_index("time", inplace=True)')
    exec(f'data_{filename[1:]}.sort_index(inplace=True)')
    
    exec(f'datum.append(data_{filename[1:]})')
df_y = pd.concat(datum, axis=1)
df_y.fillna('', inplace=True)

In [83]:
 df_y['ALL'] = df_y['@fl511_i95'] + df_y['@fl511_southeast'] + df_y['@fl511_95EXPRESS']

In [102]:
#I-95 N and S, 95 Express N and S, I-195 E and W, SR 826 N and S, SR 836 E and W, US-1 N and S
streets = ['I-95', '95 Express', 'I-195', 'SR-826', 'SR-836', 'US-1']

In [ ]:
road = pd.Series()
    for i in df_y.index:
        val = 0
        if ('I-95' in df_y['@fl511_southeast'][i]) and ('Cleared' in df_y['@fl511_southeast'][i]): 
            val = 2
        elif (street in df_y['@fl511_southeast'][i]) and (('New' or 'blocked' or 'traffic' or 'crash' or 'closed' or 'construction') in df_y['@fl511_southeast'][i]): 
            val = 1
        road[i]= val
    df_y[street] = road

In [117]:
for street in tqdm_notebook(streets):
    road = pd.Series()
    for i in df_y.index:
        val = 0
        if (street in df_y['@fl511_southeast'][i]) and ('Cleared' in df_y['@fl511_southeast'][i]): 
            val = 2
        elif (street in df_y['@fl511_southeast'][i]) and (('New' or 'blocked' or 'traffic' or 'crash' or 'closed' or 'construction') in df_y['@fl511_southeast'][i]): 
            val = 1
        road[i]= val
    df_y[street] = road

In [118]:
df_y.head(10)

,@fl511_i95,@fl511_southeast,@fl511_95EXPRESS,ALL,Miami,I-95,95 Express,I-195,SR-826,SR-836,US-1
time,,,,,,,,,,,
2019-01-01 03:00:00+00:00,Cleared: Disabled vehicle in Miami-Dade on I-9...,Cleared: Disabled vehicle in Miami-Dade on I-9...,,Cleared: Disabled vehicle in Miami-Dade on I-9...,False,2,0,0,0,0,0
2019-01-01 04:00:00+00:00,,Cleared: Crash in Miami-Dade on SR-826 south r...,,Cleared: Crash in Miami-Dade on SR-826 south r...,False,0,0,0,2,0,0
2019-01-01 05:00:00+00:00,New: Crash in Broward on I-95 north at Exit 18...,New: Crash in Broward on I-95 north at Exit 18...,,New: Crash in Broward on I-95 north at Exit 18...,False,2,0,0,2,0,0
2019-01-01 06:00:00+00:00,Cleared: Crash in Broward on I-95 north at Exi...,Cleared: Emergency vehicles in Miami-Dade on 9...,Cleared: Emergency vehicles in Miami-Dade on 9...,Cleared: Crash in Broward on I-95 north at Exi...,False,2,2,0,0,0,0
2019-01-01 07:00:00+00:00,,Updated: Crash in Miami-Dade on 95 Express nor...,Updated: Crash in Miami-Dade on 95 Express nor...,Updated: Crash in Miami-Dade on 95 Express nor...,False,0,1,0,0,0,0
2019-01-01 08:00:00+00:00,Updated: Crash in Miami-Dade on I-95 north at ...,Cleared: Emergency vehicles in Miami-Dade on 9...,Cleared: Emergency vehicles in Miami-Dade on 9...,Updated: Crash in Miami-Dade on I-95 north at ...,False,2,2,0,2,0,0
2019-01-01 09:00:00+00:00,,Cleared: Crash in Miami-Dade on SR-826 west be...,Updated: Emergency vehicles in Miami-Dade on 9...,Cleared: Crash in Miami-Dade on SR-826 west be...,False,0,2,0,2,0,0
2019-01-01 10:00:00+00:00,Cleared: Crash in Broward on I-95 north at Exi...,New: Crash in Miami-Dade on SR-112 west before...,Cleared: Emergency vehicles in Miami-Dade on 9...,Cleared: Crash in Broward on I-95 north at Exi...,False,2,2,0,0,0,0
2019-01-01 11:00:00+00:00,Updated: Crash in Broward on I-95 south before...,Updated: Multi-vehicle crash in Miami-Dade on ...,,Updated: Crash in Broward on I-95 south before...,False,0,0,0,0,0,0


In [110]:
df_y['95 Express'].value_counts()

0    4848
Name: 95 Express, dtype: int64